# Pose-Based Form Checker

Uses **pre-recorded exercise videos** to generate reference animations.
User performs in front of webcam → live feedback via stick figures (blue = reference, green/red = user).
Streamed to phone via Flask.

**Folder structure:**
```
project/
├── recordFromVideo.ipynb
└── exercises/
    ├── squat.mp4
    ├── pushup.mp4
    └── dumbbell_curls.mp4
```

In [29]:
import cv2
import mediapipe as mp
import numpy as np
import pickle
import time
import os
import glob
from flask import Flask, Response
import threading

mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

# Global frame for streaming
current_frame = None

## Helper: Calculate Angle

In [30]:
def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    ab = a - b
    bc = c - b
    cosine_angle = np.dot(ab, bc) / (np.linalg.norm(ab) * np.linalg.norm(bc) + 1e-6)
    angle = np.arccos(np.clip(cosine_angle, -1.0, 1.0))
    return np.degrees(angle)

## Extract Reference from Video (Instead of Recording Yourself)

In [31]:
def extract_reference_from_video(video_path: str, exercise_name: str, target_fps: float = 10.0):
    """
    Extracts pose landmarks from a pre-recorded video and saves as .pkl
    """
    if not os.path.exists(video_path):
        print(f"Video not found: {video_path}")
        return

    pose = mp_pose.Pose(static_image_mode=False,
                        model_complexity=1,
                        enable_segmentation=False,
                        min_detection_confidence=0.5,
                        min_tracking_confidence=0.5)

    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Cannot open video: {video_path}")
        return

    video_fps = cap.get(cv2.CAP_PROP_FPS)
    frame_interval = max(1, int(video_fps / target_fps))
    references = []
    frame_count = 0

    print(f"Extracting reference from {video_path} @ ~{target_fps} FPS...")

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        if frame_count % frame_interval == 0:
            rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = pose.process(rgb)

            if results.pose_landmarks:
                lm_list = [(lm.x, lm.y, lm.z, lm.visibility) for lm in results.pose_landmarks.landmark]
                references.append(lm_list)

        frame_count += 1

    cap.release()
    pose.close()

    # Save
    pkl_path = f"{exercise_name}.pkl"
    with open(pkl_path, 'wb') as f:
        pickle.dump(references, f)

    print(f"Saved {len(references)} reference frames → {pkl_path}")


## Auto-Discover & Extract All Videos in `exercises/` Folder

In [32]:
def extract_all_references_from_folder(folder='exercises', target_fps=10.0):
    if not os.path.exists(folder):
        print(f"Folder '{folder}' not found!")
        return

    video_extensions = ['*.mp4', '*.avi', '*.mov', '*.mkv', '*.wmv']
    video_files = []
    for ext in video_extensions:
        video_files.extend(glob.glob(os.path.join(folder, ext)))

    if not video_files:
        print("No videos found in exercises/ folder.")
        return

    print(f"Found {len(video_files)} video(s). Extracting references...\n")
    for video_path in video_files:
        name = os.path.splitext(os.path.basename(video_path))[0]
        extract_reference_from_video(video_path, name, target_fps)
    print("\nAll references extracted!")

## Run This Once: Extract All References

In [33]:
# Run this cell to process all videos in exercises/
extract_all_references_from_folder()

Found 5 video(s). Extracting references...

Extracting reference from exercises\deadlift.mp4 @ ~10.0 FPS...
Saved 35 reference frames → deadlift.pkl
Extracting reference from exercises\plank.mp4 @ ~10.0 FPS...
Saved 39 reference frames → plank.pkl
Extracting reference from exercises\push-up.mp4 @ ~10.0 FPS...
Saved 75 reference frames → push-up.pkl
Extracting reference from exercises\romanian_deadlift.mp4 @ ~10.0 FPS...
Saved 27 reference frames → romanian_deadlift.pkl
Extracting reference from exercises\squat.mp4 @ ~10.0 FPS...
Saved 51 reference frames → squat.pkl

All references extracted!


## Perform Exercise (Live Feedback + Streaming)

In [34]:
def perform_exercise(exercise_name):
    global current_frame
    
    pkl_path = f"{exercise_name}.pkl"
    if not os.path.exists(pkl_path):
        print(f"Reference not found: {pkl_path}. Run extraction first!")
        return

    with open(pkl_path, 'rb') as f:
        reference_sequence = pickle.load(f)
    
    if not reference_sequence:
        print("Empty reference sequence.")
        return

    pose = mp_pose.Pose()
    cap = cv2.VideoCapture(0)
    ref_index = 0
    animation_fps = 10
    frame_delay = 1.0 / animation_fps
    last_ref_time = time.time()
    angle_threshold = 20
    
    pose_connections = mp_pose.POSE_CONNECTIONS
    
    print(f"Starting {exercise_name}. Match the BLUE stick figure.")
    print(f"Stream: http://[YOUR-PC-IP]:5000/video  |  Press 'q' to quit.")

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        height, width, _ = frame.shape
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(rgb)
        
        black = np.zeros((height, width, 3), dtype=np.uint8)
        
        # Draw reference (BLUE)
        ref_lm = reference_sequence[ref_index]
        for conn in pose_connections:
            start, end = ref_lm[conn[0]], ref_lm[conn[1]]
            if start[3] > 0.1 and end[3] > 0.1:
                cv2.line(black,
                         (int(start[0] * width), int(start[1] * height)),
                         (int(end[0] * width), int(end[1] * height)),
                         (255, 0, 0), 2)  # BGR Blue
        
        # Draw user (GREEN if correct, RED if wrong)
        if results.pose_landmarks:
            user_lm = [(lm.x, lm.y, lm.z, lm.visibility) for lm in results.pose_landmarks.landmark]
            
            for conn in pose_connections:
                i1, i2 = conn
                r1, r2 = ref_lm[i1], ref_lm[i2]
                u1, u2 = user_lm[i1], user_lm[i2]
                
                if all(x[3] > 0.1 for x in [r1, r2, u1, u2]):
                    # Find parent joint for angle
                    parent_idx = None
                    for c in pose_connections:
                        if c[1] == i2 and c[0] != i1:
                            parent_idx = c[0]; break
                        if c[0] == i2 and c[1] != i1:
                            parent_idx = c[1]; break
                    
                    if parent_idx is not None and ref_lm[parent_idx][3] > 0.1 and user_lm[parent_idx][3] > 0.1:
                        rp, up = ref_lm[parent_idx], user_lm[parent_idx]
                        
                        ref_angle = calculate_angle((r1[0], r1[1]), (r2[0], r2[1]), (rp[0], rp[1]))
                        user_angle = calculate_angle((u1[0], u1[1]), (u2[0], u2[1]), (up[0], up[1]))
                        
                        color = (0, 255, 0) if abs(ref_angle - user_angle) < angle_threshold else (0, 0, 255)
                        
                        cv2.line(black,
                                 (int(u1[0] * width), int(u1[1] * height)),
                                 (int(u2[0] * width), int(u2[1] * height)),
                                 color, 2)
        
        # Update stream
        current_frame = cv2.imencode('.jpg', black)[1].tobytes()
        
        # Advance reference
        if time.time() - last_ref_time >= frame_delay:
            last_ref_time = time.time()
            ref_index = (ref_index + 1) % len(reference_sequence)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()

## Start Streaming Server (Run Once)

In [35]:
app = Flask(__name__)

@app.route('/')
def index():
    return '<html><body><h2>Workout Stream</h2><img src="/video" width="100%"></body></html>'

def gen():
    global current_frame
    while True:
        if current_frame is not None:
            yield (b'--frame\r\n'
                   b'Content-Type: image/jpeg\r\n\r\n' + current_frame + b'\r\n')
        time.sleep(0.01)

@app.route('/video')
def video_feed():
    return Response(gen(), mimetype='multipart/x-mixed-replace; boundary=frame')

def run_server():
    app.run(host='0.0.0.0', port=5000, threaded=True, use_reloader=False)

# Start in background
thread = threading.Thread(target=run_server, daemon=True)
thread.start()
print("Streaming server running on http://[YOUR-PC-IP]:5000")

Streaming server running on http://[YOUR-PC-IP]:5000


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.16.194.71:5000
Press CTRL+C to quit


## Perform Any Exercise

Just type the **exact filename (without extension)** from `exercises/` folder.

In [37]:
# Example: perform_exercise('squat')
# perform_exercise('pushup')
# perform_exercise('dumbbell_curls')

perform_exercise('squat')  # Change this to your video name

Starting squat. Match the BLUE stick figure.
Stream: http://[YOUR-PC-IP]:5000/video  |  Press 'q' to quit.


KeyboardInterrupt: 